<a href="https://colab.research.google.com/github/TY-Yan/Self-learning/blob/main/CrawlerPractice/Accupass%E6%B4%BB%E5%8B%95%E9%80%9A%E7%88%AC%E8%9F%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

活動通ACCUPASS

目標網站 https://www.accupass.com/?area=north

希望找到「精選活動」中所有活動的相關資訊,包含名稱、時間日期、簡介、標籤、宣傳圖

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
def Accupass():
  print('1.精選\n2.學習\n3.藝術\n4.體驗')
  target = int(input('請以數字1-4輸入要搜尋的活動類型:'))
  print('搜尋中，請稍候...')
  if target == 1:
    target = 'northfeatured'
  elif target == 2:
    target = 'learning'
  elif target == 3:
    target = 'art'
  elif target == 4:
    target = 'experience'
  else:
    print('輸入錯誤')
    
  # 從NetWok觀察到真實網址,將檔案loads出完整資料
  url = "https://api.accupass.com/v3/home/north/channel/" + target
  response = requests.get(url)
  accupass = json.loads(response.text)

  # 建立table字典裝資料
  table = {
      '活動名稱':[],
      '活動日期':[],
      '活動地點':[],
      '活動簡介':[],
      '活動標籤':[]
  }

  # for迴圈抓取在channel>tagEvents底下的資料,包含名稱.日期.簡介.標籤.圖檔網址
  for n in range(len(accupass['channel']['tagEvents'])):
    name = accupass['channel']['tagEvents'][n]['name']
    date = accupass['channel']['tagEvents'][n]['fullDateTimeStr']
    introduction = accupass['channel']['tagEvents'][n]['summary']
    tag = accupass['channel']['tagEvents'][n]['tags'][0]['name']

  # 將資料放入字典
    table['活動名稱'].append(name)
    table['活動日期'].append(date)
    table['活動簡介'].append(introduction)
    table['活動標籤'].append(tag)

  '''
  嘗試用while迴圈來抓資料,也是成功的
  i = 0
  while i < len(accupass['channel']['tagEvents']):
    print('活動名稱：', accupass['channel']['tagEvents'][i]['name'])
    print('\n活動日期：', accupass['channel']['tagEvents'][i]['fullDateTimeStr'])
    print('\n活動簡介：', accupass['channel']['tagEvents'][i]['summary'])
    print('\ntag：', accupass['channel']['tagEvents'][i]['tags'][0]['name'])
    # print(accupass['channel']['tagEvents'][i]['name'])
    print('-' * 50)
    i += 1
  '''

  # 由於活動地址並沒有顯示在真實網站中，需到各個活動網站抓地址

  # 經觀察發現活動網站由一組enevtIdNumber參數控制
  # 迴圈取得參數
  for i in range(len(accupass['channel']['tagEvents'])):
    eventID = accupass['channel']['tagEvents'][i]['eventIdNumber']
    # 參數加到網址後，取得每個活動網站的真實網址
    event_url = 'https://api.accupass.com/v3/events/'+str(eventID)
    event_response = requests.get(event_url)
    event_web = json.loads(event_response.text)
    # 找出活動地址並加入table字典
    address= event_web['address'] + '||' + event_web['addressRemark']
    table['活動地點'].append(address)

  # 將table字典寫入DataFrame，轉存成csv檔
  df = pd.DataFrame(table)
  df.to_csv('Accupass_{}.csv'.format(target), encoding='utf-8', index=False)

  # print(df) 確認無誤，結束
  return 'File get!'

In [ ]:
Accupass()

1.精選
2.學習
3.藝術
4.體驗
請以數字1-4輸入要搜尋的活動類型:3
搜尋中，請稍候...


'File get!'